Imports

In [8]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import iplot
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

device = "cpu"

making dataframe

In [9]:
def get_processed_df(path: str) -> pd.DataFrame:
    """read csv file and create dataframe
    Args:
      path: path to dataset
    Returns:
      Dataframe without NaN values
    """
    df = pd.read_csv(path, header=None)
    df.columns = ["Date",
                  "Value"]
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    if not ((df.isnull().sum()).eq(0).all()):
        df.dropna(inplace=True, ignore_index=True)

    return df
df = get_processed_df("dataset.csv")
print(df)

           Date    Value
0    2008-01-10  24.4387
1    2008-01-11  24.4796
2    2008-01-12  24.3671
3    2008-01-15  24.2913
4    2008-01-16  24.2858
...         ...      ...
3932 2023-11-28  88.7045
3933 2023-11-29  88.6102
3934 2023-11-30  88.8841
3935 2023-12-01  88.5819
3936 2023-12-02  89.7619

[3937 rows x 2 columns]


lag check

In [10]:
def time_lags(df: pd.DataFrame, n_lags: int) -> pd.DataFrame:
    """Generation of observations with a time delay

    Args:
        df (pd.DataFrame): current dataframe
        n_lags (int): count of lags

    Returns:
        pd.DataFrame: dataframe with lags {shift = 1 --> n_lags}
    """
    df_n = df.copy()
    for n in range(1, n_lags + 1):
        df_n[f"lag{n}"] = df_n["Value"].shift(n)
    df_n = df_n.iloc[n_lags:]
    return df_n

input_dim = 100

df_timelags = time_lags(df, input_dim)
print(df_timelags)

           Date    Value     lag1     lag2     lag3     lag4     lag5  \
100  2008-06-04  23.6968  23.7473  23.7384  23.6659  23.5847  23.5513   
101  2008-06-05  23.8019  23.6968  23.7473  23.7384  23.6659  23.5847   
102  2008-06-06  23.8116  23.8019  23.6968  23.7473  23.7384  23.6659   
103  2008-06-07  23.6809  23.8116  23.8019  23.6968  23.7473  23.7384   
104  2008-06-08  23.5651  23.6809  23.8116  23.8019  23.6968  23.7473   
...         ...      ...      ...      ...      ...      ...      ...   
3932 2023-11-28  88.7045  88.8133  88.1206  88.1648  87.8701  88.4954   
3933 2023-11-29  88.6102  88.7045  88.8133  88.1206  88.1648  87.8701   
3934 2023-11-30  88.8841  88.6102  88.7045  88.8133  88.1206  88.1648   
3935 2023-12-01  88.5819  88.8841  88.6102  88.7045  88.8133  88.1206   
3936 2023-12-02  89.7619  88.5819  88.8841  88.6102  88.7045  88.8133   

         lag6     lag7     lag8  ...    lag91    lag92    lag93    lag94  \
100   23.5483  23.6007  23.5758  ...  24.8917  

<ipython-input-10-0b299f3ace10>:13: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

<ipython-input-10-0b299f3ace10>:13: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

